Name: Lorenzo Ausiello

Date: 1/28/2024

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Set seed of random number generator
CWID = 20021869 
personal = CWID % 10000
np.random.seed(personal)

## Question 1 (15pt)

### Question 1.1
Use the `yfinance` package (or other method of your choice) to obtain the daily adjusted close prices for 2 different stocks.  You should have at least five years of data for both assets.  You should inspect the dates for your data to make sure you are including everything appropriately.  Create a data frame (or array) of the daily log returns both both stocks along with the lagged returns (3 lags).  Use the `print` command to display your data.

In [3]:
import yfinance
from datetime import datetime
import pandas as pd

myData= yfinance.download(['AAPL','AMZN'], datetime(2018,1,1), datetime(2024,1,1))
AAPL = myData['Adj Close']['AAPL']
AMZN = myData['Adj Close']['AMZN']
rAAPL = (np.log(AAPL) - np.log(AAPL.shift(1))).dropna()
rAMZN = (np.log(AMZN) - np.log(AMZN.shift(1))).dropna()

df = pd.DataFrame({'AAPL': rAAPL, 'AMZN': rAMZN})
print(df)

[*********************100%***********************]  2 of 2 completed
                AAPL      AMZN
Date                          
2018-01-03 -0.000174  0.012694
2018-01-04  0.004634  0.004466
2018-01-05  0.011321  0.016033
2018-01-08 -0.003721  0.014322
2018-01-09 -0.000115  0.004665
...              ...       ...
2023-12-22 -0.005563 -0.002734
2023-12-26 -0.002845 -0.000065
2023-12-27  0.000518 -0.000456
2023-12-28  0.002224  0.000261
2023-12-29 -0.005439 -0.009433

[1508 rows x 2 columns]


### Question 1.2
Split your data into training and testing sets (80% training and 20% test).

Linearly regress one of your stock returns as a function of the lagged returns (2 lags) for both stocks.
This should be of the form $r_{1,t} = \beta_0 + \beta_{1,1} r_{1,t-1} + \beta_{1,2} r_{1,t-2} + \beta_{1,3} r_{1,t-3} + \beta_{2,1} r_{2,t-1} + \beta_{2,2} r_{2,t-2} + \beta_{2,3} r_{2,t-3}$. Evaluate the performance of this model with the mean squared error on the test data.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

for i in range(1, 4):
    df[f"AAPL_lag{i}"] = df["AAPL"].shift(i)
    df[f"AMZN_lag{i}"] = df["AMZN"].shift(i)
    df[f"AMZN_lag{i}"] = df["AMZN"].shift(i)
df= df.dropna()
# Select features and target
X = df[["AAPL_lag1", "AAPL_lag2", "AAPL_lag3", "AMZN_lag1", "AMZN_lag2", "AMZN_lag3"]]
y = df["AAPL"]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model and make predictions
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)

<font color='red'>Solution:</font>

In [5]:
print("\nMean Squared Error on Test Data:", round(mse,6))


Mean Squared Error on Test Data: 0.000336


## Question 2 (35pt)

### Question 2.1
Using the same data, train/test split ratio, and consider the same regression problem as in Question 1.2.
Create a feed-forward neural network with a single hidden layer (8 hidden nodes) densely connected to the inputs.
You may choose any activation functions you wish.

In [6]:
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential()

# Hidden layer
model.add(keras.layers.Dense(units=8, input_dim=6, activation= 'relu'))

# Output layer
model.add(keras.layers.Dense(1, activation = "sigmoid"))
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 56        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 65 (260.00 Byte)
Trainable params: 65 (260.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Question 2.2
Train this neural network on the training data.  
Evaluate the performance of this model with the mean squared error on the test data.

In [7]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(),loss='mean_squared_error',metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train, y_train, epochs=500, verbose = 0)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)



10/10 [==============================] - 0s 2ms/step


<font color='red'>Solution:</font> 

In [9]:
print("\nMean Squared Error on Test Data:", round(mse,6))


Mean Squared Error on Test Data: 0.000331


### Question 2.3
Using the same train/test split and consider the same regression problem as in Question 1.2.
Train and test another neural network of your own design.

In [10]:
# Create a sequential model
model = keras.Sequential()

# Add a hidden layer with 16 units, input dimension of 6, and ReLU activation
model.add(keras.layers.Dense(units=16, input_dim=6, activation='relu'))

# Add another hidden layer with 8 units and ReLU activation
model.add(keras.layers.Dense(units=8, activation='relu'))

# Output layer with 1 unit and linear activation
model.add(keras.layers.Dense(1, activation="sigmoid"))

# Display the model summary
model.summary()

# Compile the model with Adam optimizer, mean squared error loss, and accuracy as a metric
model.compile(optimizer=keras.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])

# Train the model on the training data for 500 epochs 
model.fit(X_train, y_train, epochs=500, verbose=0)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate performance using mean squared error
mse = mean_squared_error(y_test, y_pred)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 16)                112       
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 257 (1.00 KB)
Trainable params: 257 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
10/10 [==============================] - 0s 1ms/step


<font color='red'>Solution:</font>

In [11]:
print("Mean Squared Error:", round(mse,6))

Mean Squared Error: 0.000328


## Question 3 (10pt)

## Question 3.1
Consider the same regression problem as in Question 1.2.
Of the methods considered in this assignment, which would you recommend in practice?
Explain briefly (1 paragraph) why you choose this fit. 

<font color='red'>Solution:</font> 

Among the three methods considered in this assignment, I would recommend the third method, which involves training and testing a neural network with two hidden layers. This recommendation is based on the lower Mean Squared Error (MSE) achieved by this method compared to the other two. With a MSE of 0.000327, it indicates that this neural network model provides a better fit to the test data compared to the linear regression model (MSE: 0.000336) and the feed-forward neural network with a single hidden layer (MSE: 0.000333). However, it's essential to consider factors such as model interpretability.